In [68]:
import pickle
file_path = "knowledge_base.pkl"


with open(file_path, 'rb') as file:
    knowledge_base = pickle.load(file)
# loading knowledge base

In [69]:
knowledge_base

{}

In [2]:
import spacy

/home/akash/miniconda3/envs/cleantorch/lib/python3.10/site-packages/torch/cuda/__init__.py:546: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [8]:
nlp = spacy.load('en_core_web_sm')

In [9]:
from transformers import T5ForConditionalGeneration , AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("english_to_hinglish_tokenizer")
model = T5ForConditionalGeneration.from_pretrained("my-t5-hinglish-translator")  

/home/akash/miniconda3/envs/cleantorch/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


# PUT YOU INPUT TEXT HERE #

In [ ]:
sample_passage ="Hey, bro! The local train was a total nightmare this morning. It was jam-packed, and I barely managed to squeeze in. And you won't believe it, but there was a massive traffic jam at Dadar on my way to the office. It's like everyone decided to hit the road at the same time, yaar.But, you know what, the street food here is killer. I had some amazing vada pav and cutting chai at the stall near the station. Can't beat that taste, right? And my colleagues are planning a get-together at a new rooftop bar in Bandra this Friday. You should totally join us if you're free. It's gonna be a blast"

In [ ]:
doc = nlp(sample_passage)
nouns = [token.text for token in doc if token.pos_ == "NOUN"]
nouns = set(nouns)

In [37]:

# words that might be used in their english form
sus_en_words = set()
def translate(sentence):
    sentence = "Translate English to Hinglish : " + sentence.strip().lower()
    print(f'processing {sentence}.........')
    input_ids = tokenizer(sentence , return_tensors="pt").input_ids
    outputs = model.generate(input_ids)
    output_sentence = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(f'output : {output_sentence}')
    words1 = set(sentence.lower().split())
    words2 = set(output_sentence.lower().split())

    common_words = words1.intersection(words2)
    #check whether word is a verb using spacy
    for word in common_words:
    #checking whether the word is a verb 
    #too many false positive for verb hence not using it
        if word in nouns:
            sus_en_words.add(word)


In [26]:
def process_passage(para):
    lines = para.split('.')
    for line in lines:
        translate(line)
    return sus_en_words

Put your sample passage here !!!

In [38]:
en_words = process_passage(sample_passage)

processing Translate English to Hinglish : hey, bro! the local train was a total nightmare this morning.........


/home/akash/miniconda3/envs/cleantorch/lib/python3.10/site-packages/transformers/generation/utils.py:1260: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


output : hey, bro! the local train was a total nightmare this morning
processing Translate English to Hinglish : it was jam-packed, and i barely managed to squeeze in.........
output : jam-packed ko it kare aur i barely managed to squeeze in
processing Translate English to Hinglish : and you won't believe it, but there was a massive traffic jam at dadar on my way to the office.........
output : kya won't believe it, but there was a massive traffic jam at dadar on
processing Translate English to Hinglish : it's like everyone decided to hit the road at the same time, yaar.........
output : 's like everyone decided ko hit the road at the same time, yaar
processing Translate English to Hinglish : but, you know what, the street food here is killer.........
output : but, you know what, the street food here is killer
processing Translate English to Hinglish : i had some amazing vada pav and cutting chai at the stall near the station.........
output : kya i had some amazing vada pav aur cuttin

In [39]:
en_words

{'chai',
 'food',
 'jam',
 'killer',
 'morning',
 'nightmare',
 'road',
 'stall',
 'street',
 'traffic',
 'train',
 'yaar'}

## Using google translate API

In [44]:
import requests
import os
from dotenv import load_dotenv
load_dotenv()

url = "https://google-translate1.p.rapidapi.com/language/translate/v2"
'''
payload = {
	"q": "Hello, world!",
	"target": "hi",
	"source": "en"
}
'''
print(os.getenv('CONTENT'))
headers = {
	"content-type": os.getenv('CONTENT'),
	"Accept-Encoding": os.getenv('ENCODING'),
	"X-RapidAPI-Key": os.getenv('RAPIDAPI'),
	"X-RapidAPI-Host": os.getenv('HOST')
}

#response = requests.post(url, data=payload, headers=headers)

#print(response.json())

None
{'data': {'translations': [{'translatedText': 'हैलो वर्ल्ड!'}]}}


In [72]:
def update_knowledgebase(en_words):
    for word in en_words:
        payload = {
            "q": word,
            "target": "hi",
            "source": "en"
        }
        response = requests.post(url, data=payload, headers=headers)
        print(response.json())
        print(word)
        print(response.json()['data']['translations'][0]['translatedText'])
        knowledge_base[response.json()['data']['translations'][0]['translatedText']] = word

    pickle.dump(knowledge_base, open(file_path, 'wb'))

In [73]:
update_knowledgebase(en_words)

{'data': {'translations': [{'translatedText': 'सड़क'}]}}
road
सड़क
{'data': {'translations': [{'translatedText': 'चाय'}]}}
chai
चाय
{'data': {'translations': [{'translatedText': 'ट्रैफ़िक'}]}}
traffic
ट्रैफ़िक
{'data': {'translations': [{'translatedText': 'जाम'}]}}
jam
जाम
{'data': {'translations': [{'translatedText': 'खाना'}]}}
food
खाना
{'data': {'translations': [{'translatedText': 'छोटी दुकान'}]}}
stall
छोटी दुकान
{'data': {'translations': [{'translatedText': 'सुबह'}]}}
morning
सुबह
{'data': {'translations': [{'translatedText': 'बुरा अनुभव'}]}}
nightmare
बुरा अनुभव
{'data': {'translations': [{'translatedText': 'यार'}]}}
yaar
यार
{'data': {'translations': [{'translatedText': 'हत्यारा'}]}}
killer
हत्यारा
{'data': {'translations': [{'translatedText': 'रेलगाड़ी'}]}}
train
रेलगाड़ी
{'data': {'translations': [{'translatedText': 'गली'}]}}
street
गली


In [74]:
knowledge_base

{'सड़क': 'road',
 'चाय': 'chai',
 'ट्रैफ़िक': 'traffic',
 'जाम': 'jam',
 'खाना': 'food',
 'छोटी दुकान': 'stall',
 'सुबह': 'morning',
 'बुरा अनुभव': 'nightmare',
 'यार': 'yaar',
 'हत्यारा': 'killer',
 'रेलगाड़ी': 'train',
 'गली': 'street'}

In [75]:
#now translating the passage 
response = requests.post(url, data={"q":sample_passage, "target": "hi", "source": "en"}, headers=headers)

In [76]:
edited_response = response.json()['data']['translations'][0]['translatedText']

In [77]:
edited_response

'अरे भाई! आज सुबह लोकल ट्रेन पूरी तरह से एक दुःस्वप्न थी। यह खचाखच भरा हुआ था, और मैं मुश्किल से अंदर जा पाया। और आप इस पर विश्वास नहीं करेंगे, लेकिन मेरे कार्यालय जाने के रास्ते में दादर में भारी ट्रैफिक जाम था। ऐसा लगता है कि सभी ने एक ही समय पर सड़क पर उतरने का फैसला किया है, यार। लेकिन, तुम्हें पता है, यहां का स्ट्रीट फूड जानलेवा है। स्टेशन के पास स्टॉल पर मुझे कुछ अद्भुत वड़ा पाव और कटिंग चाय मिली। उस स्वाद को हरा नहीं सकते, है ना? और मेरे सहकर्मी इस शुक्रवार को बांद्रा में एक नए रूफटॉप बार में एक साथ मिलने की योजना बना रहे हैं। यदि आप स्वतंत्र हैं तो आपको पूरी तरह से हमसे जुड़ना चाहिए। यह एक धमाका होने वाला है'

In [84]:
for word in response.json()['data']['translations'][0]['translatedText'].split():
    if word in knowledge_base.keys():
        print(word)
        edited_response = edited_response.replace(word, knowledge_base[word])

सुबह
जाम
सड़क
चाय


In [85]:
edited_response

'अरे भाई! आज morning लोकल ट्रेन पूरी तरह से एक दुःस्वप्न थी। यह खचाखच भरा हुआ था, और मैं मुश्किल से अंदर जा पाया। और आप इस पर विश्वास नहीं करेंगे, लेकिन मेरे कार्यालय जाने के रास्ते में दादर में भारी ट्रैफिक jam था। ऐसा लगता है कि सभी ने एक ही समय पर road पर उतरने का फैसला किया है, यार। लेकिन, तुम्हें पता है, यहां का स्ट्रीट फूड जानलेवा है। स्टेशन के पास स्टॉल पर मुझे कुछ अद्भुत वड़ा पाव और कटिंग chai मिली। उस स्वाद को हरा नहीं सकते, है ना? और मेरे सहकर्मी इस शुक्रवार को बांद्रा में एक नए रूफटॉप बार में एक साथ मिलने की योजना बना रहे हैं। यदि आप स्वतंत्र हैं तो आपको पूरी तरह से हमसे जुड़ना चाहिए। यह एक धमाका होने वाला है'